# Learn the basics of Blockchain with Julia
## Part 1 - Cryptography

We will start out our journey using Blockchain in Julia learning about some simple cryptographic functions (of which there are many flavors and variants). Cryptography is the back bone of blockchain and while that may not be clear now, as we work through some future examples, you will start to see why it is so important.

The first example we will look at uses SHA 256, which quite simply takes some data and generates an almost unique signature (more on that later) for that data. It is worth noting that SHA 256 and other cryptographic functions are not encrypting the data. Encrypting means we are able to reverse the process with some function. That is not the case with hash functions.  Want to learn more about the inner workings of SHA-256? Check out: https://qvault.io/cryptography/how-sha-2-works-step-by-step-sha-256/

In [1]:
# We will be using https://github.com/JuliaCrypto/SHA.jl
using SHA

So, let's play around with some cryptographic functions. When we run the `sha256` function below, pay attention to the output.

In [2]:
sha256("test")

32-element Vector{UInt8}:
 0x9f
 0x86
 0xd0
 0x81
 0x88
 0x4c
 0x7d
 0x65
 0x9a
 0x2f
 0xea
 0xa0
 0xc5
    ⋮
 0x2b
 0x0b
 0x82
 0x2c
 0xd1
 0x5d
 0x6c
 0x15
 0xb0
 0xf0
 0x0a
 0x08

You can see that the output is a vector of Unsigned Int's, 32 of them to be exact. This makes perfect sense given that SHA256 has a 32-bit output. But the above is kind of hard to look at, so let's convert that from bytes to hex... 

In [5]:
bytes2hex(sha256("test"))

"9f86d081884c7d659a2feaa0c55ad015a3bf4f1b2b0b822cd15d6c15b0f00a08"

In [4]:
typeof(bytes2hex(sha256("test")))

String

Here you can see the result of converting bytes to hex and that hex is represented using a String in Julia. So far we have been using the SHA package in Julia, but we are reaching the end of its use. We will now look at a more comprehensive package called Nettle.jl and see why it is likely what we will want to use moving forward.

In [7]:
# Note that the best practive is to have all using's at the top of the file.
using Nettle

In [9]:
h = Hasher("sha256")
Nettle.update!(h, "this is a test")
hexdigest!(h)

"2e99758548972a8e8822ad47fa1017ff72f06f3ff6a016851f45c398732bc50c"

Above, we create a `Hasher`, set the encoding to `sha256`, and then call the update function to get the hash value of "this is a test". Lastly, we call the `hexdigest!` function to get the actual hex representation of the string. Nettle also allows for other hash types like `SHA512` as seen below.

In [10]:
hexdigest("sha512", "this is a test")

"7d0a8468ed220400c0b8e6f335baa7e070ce880a37e2ac5995b9a97b809026de626da636ac7365249bb974c719edf543b52ed286646f437dc7f810cc2068375c"

A quick word on why this all matters: hash functions are the backbone of blockchain. This is because there is the need for unique identifiers (hash's) of various objects/strings across the blockchain ecosystem. For example, an address of someone in a blockchain network can be hashed. We don't necessarily want everyone to know the address of all other addresses that exist, so we we used 1, 2, 3, etc to label addresses, there would be a lot of issues that come up. Besides addresses, there are many other use-cases for hash function that we will cover in more details later on. 

## Encryption and Decryption
Now that we have played around a little with some of the cryptographic functions in the Nettle and SHA, let's dive into the ability to encrypt information using Nettle.jl.

In [22]:
key = SHA.sha256("This is my key")
enc = Encryptor("AES256", key)

AES256 Encryption state

Above we created our key using sha256 and then created an encryptor using the `AES256` standard.

In [25]:
# Create a 16 byte string.
plaintext = "This is a secret"
ciphertext = encrypt(enc, plaintext)

16-element Vector{UInt8}:
 0xac
 0x6a
 0xb9
 0x00
 0x82
 0xc1
 0x8e
 0x3f
 0x68
 0x77
 0xc6
 0xab
 0x68
 0x67
 0xf3
 0x30

In [27]:
dec = Decryptor("AES256", key)
deciphertext = decrypt(dec, ciphertext)

16-element Vector{UInt8}:
 0x54
 0x68
 0x69
 0x73
 0x20
 0x69
 0x73
 0x20
 0x61
 0x20
 0x73
 0x65
 0x63
 0x72
 0x65
 0x74

In [28]:
# Check to make sure the values are the same...
Vector{UInt8}(plaintext) == deciphertext # no bytestring

true

In [30]:
Vector{UInt8}("This is the wrong secret") == deciphertext # no bytestring

false

In [32]:
String(deciphertext)

"This is a secret"

Above we can see that after encryption and decryption, we still have the origional plain text value.

In the examples above, there were a bunch of restrictions on the cipher text and private key. Given those constraints, let's look at another tool which we can use to generate key pairs.  

In [33]:
using ECC

In [34]:
privateKey = ECC.PrivateKey(10)

PrivateKey(10, scep256k1 Point(𝑥,𝑦):
a0434d9e47f3c86235477c7b1ae6ae5d3442d49b1943c2b752a68e2a47e247c7,
893aba425419bc27a3b6c7e693a24c696f794c2ed877a1593cbee53b037368d7)

In [40]:
signature = ECC.pksign(privateKey, 10)

scep256k1 signature(𝑟, 𝑠):
b6a40dd053534268a5183de5207368e284e9ff7452144e08b7d3df2092be7e04,
4515d3874d94387f084050d658f951b2696574a61ed4ba29b29bf4cef8a01b8e

In [41]:
ECC.verify(privateKey.𝑃, 10, signature)

true

Let's break down quickly what we did in the above lines of code. First, we created a private key using some int to start (required simply due to the implementation). Then, we generated a signature and verified that the signature generated was correct for a specific number, 10.

In [42]:
ECC.verify(privateKey.𝑃, 25, signature)

false

Here, we can see that since we are using 25, the verify function now returns false. It is worth noting that the ECC package has a few limitations. One of which is that you can only use ints in these cases despite the docs saying that the functions accept generic data. Another limitation is that there is no ability to generate a public key, which is a core part of blockchain and how people securely interact on the internet. 

Hopefully this notebooks provides some insight into the state of some of the various Cryptographic functionalities available in Julia at the present moment. In the upcoming notebooks, we will dive into various other blockchain related ideas.